In [1]:
import os
import re
import datetime
import csv
global HOME_PATH
global LOG_PATH
global MAS_FILE
HOME_PATH = '/home/jovyan/'
LOG_PATH = f"{HOME_PATH}datasets/Zcsvlogs/"
DAT_PATH = f"{HOME_PATH}datasets/Zcsvlogs/Parfile/"
MAS_FILE = {}

In [2]:
def main():
    files = os.listdir(LOG_PATH)
    outcsv = f'{DAT_PATH}transit1.txt'
    results = []
    print(len(MAS_FILE))
    for file in files:
        #m = re.findall(r'^(?P<ymd>\d{8})(\d{6})_(\w+)_(\w+)_(\w+)_(\w+)\.LOG', file) 
        #y = '20220418'
        #d = '091633'
        y = '\d{8}'
        #d = '1011\d{2}'
        d = '\d{6}'
        s = \
        f'^(?P<ymd>{y})' + \
        f'(?P<hms>{d})_' +  \
        '(?P<zcsv>\w+)_' +  \
        '(?P<apid>\w+)_' +  \
        '(?P<ftpid>\w+)_' + \
        '(?P<filecode>.+)\.LOG'
        
        m = re.match(s, file)
        if m :
            #print(file)
            res = parse(
                file, 
                m.group('ymd'),
                m.group('hms'),
                m.group('zcsv'),
                m.group('apid'),
                m.group('ftpid'),
                m.group('filecode')
            )
            for r in res:
                results.append(r)
                
    #print(results)
    h1 = [";1","2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]
    h2 = [";ID", "ZIP", "Folder", "File", "FTP", "Trig", "Rows", "Secs", "FileNo", "Start", "End"]
    with open(outcsv, 'w') as f:       
        writer = csv.DictWriter(f, h1, delimiter='\t')
        writer.writeheader()
        writer = csv.DictWriter(f, h2, delimiter='\t')
        writer.writeheader()
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(results)

In [3]:
def master1(key):
    mas1 = {
        "D:\\Ftp\\ap00234\\ftp5000\\Recv\\HostLink\\"   : {"ftp": "/ftp5000"                , "operate":"Recv"} ,
        "D:\\Ftp\\ap00274\\ftp1100\\Recv\\HostLink\\"   : {"ftp": "/ftp1100/Recv/HostLink"  , "operate":"Recv"},
        "D:\\Ftp\\ap00274\\ftp1100\\Send\\ServerLink\\" : {"ftp": "/ftp1100/Send/ServerLink", "operate":"Send"},
        "D:\\Ftp\\ap00411\\JKFTP\\HostFrom\\HostLink\\" : {"ftp": "/JKFTP/HostFrom\HostLink", "operate":"Recv"},
        "D:\\Ftp\\ap00411\\JKFTP\\HostTo\\ServerLink\\" : {"ftp": "/JKFTP/HostTo/ServerLink", "operate":"Send"},
        "D:\\Ftp\\ap00476\\ftp6000\\Recv\\HostLink\\"   : {"ftp": "/ftp6000"                , "operate":"Recv"},
        "D:\\Ftp\\ap00476\\ftp6001\\Recv\\HostLink\\"   : {"ftp": "/ftp6001/Recv/HostLink"  , "operate":"Recv"},
        "D:\\Ftp\\ap00476\\ftp6001\\Send\\ServerLink\\" : {"ftp": "/ftp6001/Send/ServerLink", "operate":"Send"},
    }
    return mas1[key]["ftp"], mas1[key]["operate"]

In [4]:
def filemas(key, value):
    bln = key in MAS_FILE
    res = False
    if not bln:
        #print(f'{key} {value}')
        MAS_FILE[key] = value
        res = True
    return res

In [5]:
def parse(file, ymd, hms, zcsv, apid, ftpid,filecode):
    path = f'{LOG_PATH}{file}'
    results = []
    with open(path, encoding='ShiftJIS') as f:
        l = f.readlines()
        okey = '$$$'
        nkey = ''
        start = 0
        end = 0
        dfc = 0
        dfs = 0
        #print(f'{file} %d' % (len(l)))
        for line in l:
            #--------------------------------------------------------------
            # ファイルパス出現パターン
            # 開始 0→1
            s1 = 'のファイルをバックアップします'
            m = re.search(s1, line)
            #--------------------------------------------------------------
            # 変換開始、ファイルパス出現パターン
            if m:
                start = 1
                #----------------------------------------------------------
                # ファイルパス取得パターン
                s = '\[(\d{4}/\d{2}/\d{2})\s+(\d{1,2}:\d{2}:\d{2})\.\d{2}\]\s+(.*)' + s1
                m = re.match(s, line)
                #----------------------------------------------------------
                dymd1 = m.group(1)
                dhms1 = m.group(2)
                dpath = m.group(3)
                #----------------------------------------------------------
                # ファイル名取得パターン
                m = re.search('[^\\\\]+$', dpath)
                #----------------------------------------------------------
                if m:
                    dfc += 1
                    dfile = m.group(0)
                    dzip = f'{zcsv}_{apid}_{ftpid}_{dfile}.ZIP'
                    s = f'{zcsv}'
                    m = re.search(s, dfile)
                    dkbn = 0
                    if m:
                        dkbn = 1
                        did = dfile
                #----------------------------------------------------------
                # ファイル名取得パターン
                m = re.search('^.*\\\\', dpath)
                #----------------------------------------------------------
                if m:
                    ddir = m.group(0)
                
            #--------------------------------------------------------------
            # ファイル出力完了出現パターン
            s1 = '出力レコード数'
            m = re.search(s1, line)
            #--------------------------------------------------------------
            if m:
                #----------------------------------------------------------
                # 変換件数取得パターン
                s = '^\s+' + s1 + '\s+=\s+(\d+)'
                m = re.search(s, line)
                #----------------------------------------------------------
                if m:
                    dcount = m.group(1)
            #--------------------------------------------------------------
            # 変換終了出現パターン
            s1 = 'の変換が正常に完了しました'
            m = re.search(s1, line)
            #--------------------------------------------------------------            
            if m:
                end = 1
                s = '\[(\d{4}/\d{2}/\d{2})\s+(\d{1,2}:\d{2}:\d{2})\.\d{2}\]\s+(.*)' + s1
                m = re.search(s, line)
                #----------------------------------------------------------
                dymd2 = m.group(1)
                dhms2 = m.group(2)
                dpath2 = m.group(3)
            
            if (start, end) == (1, 1):
                if filemas(f'{did}_{dfc}', dfc):
                    dts = datetime.datetime.strptime(f'{dymd1} {dhms1}' , '%Y/%m/%d %H:%M:%S')
                    dte = datetime.datetime.strptime(f'{dymd2} {dhms2}' , '%Y/%m/%d %H:%M:%S')
                    secs = abs(dts - dte).seconds
                    ftpcd , ftpope = master1(ddir)
                    results.append([did, dzip, ftpope, dfile, ftpcd, dkbn, dcount, secs, dfc,dts,dte])
                    #print(f'{did} {dzip} {ftpope} {dfile} {ftpcd} {dkbn} {dfc}')
                
                start = 0
                end = 0
    #print(results)
    return results

In [6]:
MAS_FILE = {}
main()

0
